<a href="https://colab.research.google.com/github/saadkhi/Side/blob/main/Script2_for_fintune_w_LLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Packages & Libraries**

In [1]:
!pip install faiss-cpu
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 449.4 kB/s eta 0:00:0000:0100:02
Defaulting to user installation because normal site-packages is not writeable


# **Dataset Info**

In [2]:
import json

docs = []
with open("dataset/dataset.jsonl", "r") as f:
    for line in f:
        row = json.loads(line)
        text = f"Q: {row['prompt']}\nA: {row['completion']}"
        docs.append(text)

print(len(docs))

11171


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # lightweight & fast

doc_embeddings = embedder.encode(docs, convert_to_numpy=True, show_progress_bar=True)

# Build FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

print("Index size:", index.ntotal)

/home/saad/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches:   0%|          | 0/350 [00:00<?, ?it/s]/home/saad/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 350/350 [04:10<00:00,  1.39it/s]

Index size: 11171


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "microsoft/phi-3-mini-4k-instruct"  # or llama2-chat if you have GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`

In [ ]:
def is_database_query(query: str) -> bool:
    keywords = ["sql", "nosql", "database", "query", "table", "index", "join"]
    return any(k in query.lower() for k in keywords)

In [ ]:
# def chatbot(query):
#     # Step 1: Check domain relevance
#     if not is_database_query(query):
#         return "I only answer database-related queries."

#     # Step 2: Retrieve top matches
#     q_embedding = embedder.encode([query], convert_to_numpy=True)
#     D, I = index.search(q_embedding, k=3)  # top 3

#     # Step 3: Check threshold
#     if D[0][0] > 0.6:
#         # High similarity, use dataset
#         context = "\n".join([docs[i] for i in I[0]])
#     else:
#         # No good dataset match → fallback to model
#         context = ""

#     # Step 4: Build prompt
#     prompt = f"""
#     You are a SQL/NoSQL expert.
#     Context: {context if context else "No context found."}
#     Question: {query}

#     Instructions:
#     - If context is provided, answer only from it.
#     - If no context, but question is still about databases, answer from your own knowledge.
#     - If not database related, say: "I only answer database-related queries."
#     """

#     response = llm(prompt, max_new_tokens=300, do_sample=True)[0]['generated_text']
#     return response

In [ ]:
# print("💬 Database Chatbot is ready! (type 'exit' to quit)\n")

# while True:
#     query = input("You: ").strip()
#     if query.lower() in ["exit", "quit", "bye"]:
#         print("Bot: Goodbye! 👋")
#         break

#     response = chatbot(query)
#     print(f"Bot: {response}\n")


💬 Database Chatbot is ready! (type 'exit' to quit)

You: write query to merge 2 columns in a table.
Bot: 
    You are a SQL/NoSQL expert.
    Context: Q: Use SQL UNION to combine results
A: SELECT col FROM table1 UNION SELECT col FROM table2;
Q: Write a SQL query with UNION
A: SELECT column FROM table1 UNION SELECT column FROM table2;
Q: Write a SQL query to concatenate strings
A: SELECT CONCAT(column1, column2) FROM table_name;
    Question: write query to merge 2 columns in a table.

    Instructions:
    - If context is provided, answer only from it.
    - If no context, but question is still about databases, answer from your own knowledge.
    - If not database related, say: "I only answer database-related queries."
    
    Question: write SQL query to merge two columns in a table.

    Answer:

    To merge two columns into a single column in SQL, you can use the concatenation function, which varies depending on the SQL dialect you're using. For example, in PostgreSQL, you can us

In [ ]:
def chatbot(query):
    # Step 1: Check domain relevance
    if not is_database_query(query):
        return "I only answer database-related queries."

    # Step 2: Retrieve top matches
    q_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(q_embedding, k=3)  # top 3

    # Step 3: Check threshold
    if D[0][0] > 0.6:
        context = "\n".join([docs[i] for i in I[0]])
    else:
        context = ""

    # Step 4: Build prompt
    prompt = f"""
    You are a SQL/NoSQL expert.
    Context: {context if context else "No context found."}
    Question: {query}

    Answer:
    """

    # Run through model
    raw_output = llm(prompt, max_new_tokens=300, do_sample=True)[0]['generated_text']

    # Extract only the part after "Answer:"
    if "Answer:" in raw_output:
        answer = raw_output.split("Answer:", 1)[1].strip()
    else:
        answer = raw_output.strip()

    return answer


In [ ]:
print("💬 Database Chatbot is ready! (type 'exit' to quit)\n")

while True:
    query = input("You: ").strip()
    if query.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye! 👋")
        break

    response = chatbot(query)
    print(f"Bot: {response}\n")

💬 Database Chatbot is ready! (type 'exit' to quit)

You: write query to merge 2 columns in a table.
Bot: You can merge two columns in a SQL table using the concatenation operator. Here's an example query that merges two columns, `first_name` and `last_name`, into a single column named `full_name` in a table called `users`:

    ```sql
    SELECT first_name ||'' || last_name AS full_name
    FROM users;
    ```

    This will return a result set with a single `full_name` column, containing the combined values of `first_name` and `last_name` from each row, separated by a space. The `||` operator is used for string concatenation in SQL, but the specific syntax can vary depending on the database system you're using (e.g., `+` in Oracle or PostgreSQL).

You: Tell me history of karachi
Bot: I only answer database-related queries.

You: What is CAP theorem?
Bot: I only answer database-related queries.

You: What is CAP theorem in databases?
Bot: The CAP theorem, also known as Brewer's theorem